# __311 Service Requests in Chicago__

## Import dependencies

In [1]:
import altair as alt
import pandas as pd
import geopandas as gpd

## Helper Functions

In [2]:
def show_col_row_count(df):

    row, col = df.shape
    msg = "The dataframe currently has {} rows and {} columns."
    print(msg.format(row, col))

def drop_non_numeric(df, columns, verbose=False):

    if verbose:
        rows, _ = df.shape
        col_msg = "Removing all non-numeric rows for the following columns: {}."
        print(col_msg.format(columns))
    for col in columns:
        df = df[pd.to_numeric(df[col], errors="coerce").notnull()]
        if verbose:
            new_rows, _ = df.shape
            print("Removed {} rows from {}.".format(rows - new_rows, col))
            rows = new_rows
    return df

def drop_na(dataframe, columns, verbose=False):

    if verbose:
        rows, _ = dataframe.shape
        col_msg = "Removing all rows with NAs for the following columns: {}."
        print(col_msg.format(columns))
    df = dataframe.dropna(subset = columns)
    if verbose:
        new_rows, _ = dataframe.shape
        print("Removed {} rows.".format(rows - new_rows))
    return df

def save_columns(dataframe, columns):

    dataframe = dataframe[columns]
    return dataframe

def df_to_csv(dataframe, filename, verbose=False):

    try:
        dataframe.to_csv(filename, index=False)
        if verbose:
            print("Successfully saved to {}.".format(filename))
    except:
        if verbose:
            print("Failed to save to {}.".format(filename))

def extract_census_tract(row):
    return row.split(",")[0]

def evaluateForBivariate(row):
    # Adapted from the slack channel (Credit to Andrew McNutt)
    colorMatrix = [
        ["#64acbe", "#627f8c", "#574249"],
        ["#b0d5df", "#ad9ea5", "#985356"],
        ["#e8e8e8", "#e4acac", "#c85a5a"],
    ]
    xBoundaries = [1165.02, 1959.06]
    xIdx = 0
    if row["Number_of_Requests"] < xBoundaries[0]:
        xIdx = 0
    elif row["Number_of_Requests"] < xBoundaries[1]:
        xIdx = 1
    else:
        xIdx = 2
    yBoundaries = [8200.15, 12498.46]
    yIdx = 0
    if int(row["Household_Income"]) < yBoundaries[0]:
        yIdx = 2
    elif int(row["Household_Income"]) < yBoundaries[1]:
        yIdx = 1
    else:
        yIdx = 0

    return colorMatrix[yIdx][xIdx]

## Load the dataset - `311_Service_Requests.csv`

In [3]:
data_dir = "./data/311_Service_Requests_Chicago.csv"
dtype_dict = {
    "STREET_NUMBER": "object",
    "LEGACY_SR_NUMBER": "object",
    "PARENT_SR_NUMBER": "object",
    "SANITATION_DIVISION_DAYS": "object",
    }
df = pd.read_csv(data_dir, dtype=dtype_dict)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df["CREATED_DATE"] = pd.to_datetime(df["CREATED_DATE"], format='%m/%d/%Y %I:%M:%S %p')
df["CREATED_YEAR"] = df["CREATED_DATE"].dt.year
df["CLOSED_DATE"].fillna("01/16/2020 10:20:00 PM", inplace = True) 
df["CLOSED_DATE"]= pd.to_datetime(df["CLOSED_DATE"], format='%m/%d/%Y %I:%M:%S %p')
df["CLOSED_YEAR"] = df["CLOSED_DATE"].dt.year
df["DAYS_TAKEN"] = df["CLOSED_DATE"] - df["CREATED_DATE"]
df["DAYS_TAKEN"] = df["DAYS_TAKEN"].dt.days

In [5]:
df.sample(n = 1) 

,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,LAST_MODIFIED_DATE,CLOSED_DATE,STREET_ADDRESS,CITY,...,CREATED_DAY_OF_WEEK,CREATED_MONTH,X_COORDINATE,Y_COORDINATE,LATITUDE,LONGITUDE,LOCATION,CREATED_YEAR,CLOSED_YEAR,DAYS_TAKEN
1631758,SR19-00936264,Graffiti Removal Request,GRAF,Streets and Sanitation,Completed,2018-10-30 14:10:25,05/17/2019 12:25:34 PM,2018-10-31 14:15:48,2655 W CERMAK RD,CHICAGO,...,3,10,1.158780e+06,1.889279e+06,41.851922,-87.692726,"(41.851921903115795, -87.69272618195643)",2018,2018,1


In [6]:
show_col_row_count(df)

The dataframe currently has 2362297 rows and 40 columns.


In [7]:
df = drop_non_numeric(df, ["ZIP_CODE"], verbose=True)

Removing all non-numeric rows for the following columns: ['ZIP_CODE'].
Removed 17961 rows from ZIP_CODE.


In [8]:
df = drop_na(df, ["ZIP_CODE", "CREATED_DATE", "WARD", "LOCATION", "LATITUDE", "LATITUDE"], verbose=True)

Removing all rows with NAs for the following columns: ['ZIP_CODE', 'CREATED_DATE', 'WARD', 'LOCATION', 'LATITUDE', 'LATITUDE'].
Removed 0 rows.


In [9]:
columns = [
    "SR_NUMBER", "SR_TYPE", "SR_SHORT_CODE",
    "OWNER_DEPARTMENT","STATUS", "CREATED_DATE",
    "CLOSED_DATE", "ZIP_CODE", "DUPLICATE",
    "LEGACY_RECORD", "WARD", "LATITUDE",
    "LONGITUDE", "LOCATION",
    "DAYS_TAKEN", "CREATED_YEAR", "CLOSED_YEAR",
    "CREATED_HOUR",	"CREATED_DAY_OF_WEEK", "CREATED_MONTH"
]
df = save_columns(df, columns)

In [10]:
df.sample(n=1)

,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,ZIP_CODE,DUPLICATE,LEGACY_RECORD,WARD,LATITUDE,LONGITUDE,LOCATION,DAYS_TAKEN,CREATED_YEAR,CLOSED_YEAR,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH
324983,SR19-01929310,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-07-12 11:19:42,2019-07-12 11:19:42,60612,False,False,28.0,41.871831,-87.679846,"(41.871831277993564, -87.67984621876099)",0,2019,2019,11,6,7


In [11]:
show_col_row_count(df)

The dataframe currently has 2325604 rows and 20 columns.


In [12]:
# Save the dataset

# Dataset for 2018/07/01 - 2018/12/31
df_2018 = df[(df['CREATED_DATE'] > '2018-07-01 00:00:00') & (df['CREATED_DATE'] <= '2018-12-31 23:59:59')]
filename_2018 = "./data/cleaned_311_2018.csv"
df_to_csv(df_2018, filename_2018, verbose=True)
# Dataset for 2019/07/01 - 2019/12/31
df_2019 = df[(df['CREATED_DATE'] > '2019-07-01 00:00:00') & (df['CREATED_DATE'] <= '2019-12-31 23:59:59')]
filename_2019 = "./data/cleaned_311_2019.csv"
df_to_csv(df_2019, filename_2019, verbose=True)
# Entire cleaned dataset
filename = "./data/cleaned_311.csv"
df_to_csv(df, filename, verbose=True)

Successfully saved to ./data/cleaned_311_2018.csv.
Successfully saved to ./data/cleaned_311_2019.csv.
Successfully saved to ./data/cleaned_311.csv.


## Create a dataset for Census Tract (Number of Requests and Income Level)



In [13]:
# Load Census Tract Polygon dataset
tract_dir = "./data/census_tract_boundary/tracts.geojson"
tracts = gpd.read_file(tract_dir)
tracts.crs = 'epsg:4326'
tracts.sample(n=1)

,statefp10,name10,commarea_n,namelsad10,commarea,geoid10,notes,tractce10,countyfp10,geometry
575,17,309,77,Census Tract 309,77,17031030900,,030900,031,"MULTIPOLYGON (((-87.67259 41.97613, -87.67367 ..."


In [14]:
cleaned_311 = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LONGITUDE, df.LATITUDE))

In [15]:
cleaned_311.sample(n=1)

,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,ZIP_CODE,DUPLICATE,LEGACY_RECORD,...,LATITUDE,LONGITUDE,LOCATION,DAYS_TAKEN,CREATED_YEAR,CLOSED_YEAR,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH,geometry
2117143,SR19-02949173,Snow – Uncleared Sidewalk Complaint,SWSNOREM,CDOT - Department of Transportation,Completed,2019-11-14 18:36:43,2019-11-21 09:24:13,60641,True,False,...,41.956306,-87.73215,"(41.95630642007092, -87.73215041449315)",6,2019,2019,18,5,11,POINT (-87.73215 41.95631)


In [ ]:
cleaned_311.crs = "epsg:4326"
cleaned_311 = cleaned_311[cleaned_311.geometry.type == 'Point']
tracts_311 = gpd.sjoin(cleaned_311, tracts, how="inner", op='intersects')

In [ ]:
tracts_311.sample(n=1)

In [ ]:
tracts_count = tracts_311.groupby(["namelsad10"]).size().reset_index()
tracts_count = tracts_count.rename(columns={0: "Number_of_Requests"})

In [ ]:
tracts_count.sample(n=1)

In [ ]:
# load income dataset
income_dir = "./data/income_by_census_tract/income_2018_by_census_tract.csv"
income_df = pd.read_csv(income_dir)
income_df.sample(n=1)

In [ ]:
# Extract Census Tract
income_df["NAME"] = income_df["NAME"].apply(lambda x : extract_census_tract(x))

# Save Estimate!!Households!!Mean income (dollars)
columns = ["NAME", "S1901_C01_013M"]
income_df = save_columns(income_df, columns)
income_df = income_df.rename(columns={"NAME": "namelsad10", "S1901_C01_013M": "Household_Income"})

In [ ]:
# Merge it with tracts_count
tracts_count = tracts_count.merge(income_df, on='namelsad10', how='left')
tracts_count = drop_non_numeric(tracts_count, ["Household_Income"], verbose=True)

In [ ]:
tracts_count.sample(n=1)

In [ ]:
# Save the merged dataset
tracts_311_count_filename = "./data/tracts_count.csv"
df_to_csv(tracts_count, tracts_311_count_filename, verbose=True)

In [ ]:
tracts_count.sample(n=1)

In [ ]:
tracts_count["color"] = tracts_count.apply(lambda x : evaluateForBivariate(x), axis = 1)

In [ ]:
tracts_count.sample(n=1)

In [ ]:
# Save the merged dataset
tracts_311_count_filename = "./data/tracts_count.csv"
df_to_csv(tracts_count, tracts_311_count_filename, verbose=True)

In [ ]:
tracts_311_pothole = tracts_311[(tracts_311['SR_TYPE'] == "Pothole in Street Complaint")]

In [ ]:
tracts_311_pothole.sample(n=1)

In [ ]:
# Save Pothole dataset
pothole_filename = "./data/tracts_311_pothole.csv"
df_to_csv(tracts_311_pothole, pothole_filename, verbose=True)

In [ ]:
tracts_311_pothole.shape

In [ ]:
tracts_311_call = tracts_311[(tracts_311['SR_TYPE'] == "311 INFORMATION ONLY CALL")]

# Save 311 call dataset
call_filename = "./data/tracts_311_call.csv"
df_to_csv(tracts_311_call, call_filename, verbose=True)

In [ ]:
tracts_311_call.sample(n=1)